###**Connecting to Kaggle and loading data**

In [0]:
from google.colab import files
files.upload()

In [0]:
!pip install -q kaggle

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c facial-keypoints-detection

In [0]:
!unzip \test.zip

In [0]:
!unzip \training.zip

### **Importing Necessary Packages**

In [0]:
%tensorflow_version 1.x
import utilities as utils
import DataAugmentation

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
import os

import tensorflow as tf
from tensorflow.keras.utils import plot_model
from sklearn.utils import shuffle
from PIL import Image, ImageFile
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
ImageFile.LOAD_TRUNCATED_IMAGES = True
%matplotlib inline

###**Cleaning, Normalizing and Loading Data**

In [0]:
X_train, y_train = utils.loadData('train', drop_null=True)

In [0]:
print ("Shape of X:",X_train.shape, "Min of X:", X_train.min(), "Max of X:", X_train.max())
print ("Shape of y:",y_train.shape, "Min of y:", y_train.min(), "Max of y:", y_train.max())

**Exploring Dataset**

In [0]:
fig = plt.figure(figsize=(10,10))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.1, wspace=0.1)

images = np.random.choice(range(X_train.shape[0]), 16, replace=False)
for i in range(16):
  axis = fig.add_subplot(5,4, i+1, xticks=[], yticks=[])
  utils.plotSample(X_train[i], y_train[i], axis)

### **Model 1**

####**Defining Model**

In [0]:
def Model_1():
  model = tf.keras.Sequential()

  model.add(tf.keras.layers.Conv2D(filters=32,
                                   kernel_size=3, 
                                   activation='relu', 
                                   input_shape=(96,96,1)
                                   ))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
  model.add(tf.keras.layers.BatchNormalization())

  model.add(tf.keras.layers.Conv2D(filters=128,
                                   kernel_size=4,
                                   activation='relu',
                                   ))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dropout(0.1))

  model.add(tf.keras.layers.Conv2D(filters=256,
                                   kernel_size=5,
                                   activation='relu',
                                   ))
  # model.add(tf.keras.layers.GlobalAveragePooling2D())
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(3,3)))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dropout(0.2))

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(32, activation='relu'))
  model.add(tf.keras.layers.BatchNormalization())
#   model.add(tf.keras.layers.Dropout(0.1))
  model.add(tf.keras.layers.Dense(30))

  return model

In [0]:
model = Model_1()
base_learning_rate = 1e-4

model.compile(
    loss = 'mse',
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate),
    metrics = ['accuracy'] 
)

model.summary()

**Model Architecture**

In [0]:
plot_model(
    model, 
    to_file='model.png', 
    show_layer_names=True, 
    show_shapes=True)

####**Training and Checking the Model**

In [0]:
epochs = 300 #epochs before the loss starts increasing
batch_size = 32

history = model.fit(
    X_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.2,
    verbose=2
)

**Learning Rate Check**
A check for the optimal curve, monotonically decreasing loss without regularization

In [0]:
plt.figure()

loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['acc']
val_acc = history.history['val_acc']

plt.plot(loss,linewidth=2, label="training loss")
plt.plot(val_loss, linewidth=2, label="validation loss")

plt.legend()
plt.yscale("log")
plt.xlabel("epoch")
plt.ylabel("log loss")
plt.show()

plt.plot(acc, linewidth=2, label='training accuracy')
plt.plot(val_acc, linewidth=2, label='validation accuracy')
plt.legend(loc='lower right')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.show()

**Continue training with new learning rate**

In [0]:
more_epochs = 500
new_learning_rate = 1e-6

model.compile(
    loss = 'mse',
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=new_learning_rate),
    metrics = ['accuracy']
)

last_epoch = history.epoch[-1] + 1
history = model.fit(
    X_train,
    y_train,
    epochs=more_epochs+last_epoch,
    initial_epoch = last_epoch,
    batch_size=batch_size,
    validation_split=0.2,
    verbose=2
)

**Learning Rate Check** A check for the optimal curve, monotonically decreasing loss without regularization

In [0]:
plt.figure()

loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['acc']
val_acc = history.history['val_acc']

plt.plot(loss,linewidth=2, label="training loss")
plt.plot(val_loss, linewidth=2, label="validation loss")

plt.legend()
plt.yscale("log")
plt.xlabel("epoch")
plt.ylabel("log loss")
plt.show()

plt.plot(acc, linewidth=2, label='training accuracy')
plt.plot(val_acc, linewidth=2, label='validation accuracy')
plt.legend(loc='lower right')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.show()

####**Testing the Model**

In [0]:
X_test, _ = utils.loadData()

In [0]:
y_test = model.predict(X_test)

In [0]:
fig = plt.figure(figsize=(10,10))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.3, wspace=0.1)

images = np.random.choice(range(X_test.shape[0]), 16, replace=False)

for i in range(16):
  axis = fig.add_subplot(5,4, i+1, xticks=[], yticks=[])
  utils.plotSample(X_test[images[i]], y_test[images[i]], axis, "Test Prediction {}".format(i+1))

###**Adding Data Augmentation**

#####**Preparing Augmented Data**

In [0]:
augmentor = DataAugmentation.Augmentor()

In [0]:
h_flipped_images, h_flipped_keypoints = augmentor.flip(X_train, y_train)
v_flipped_images, v_flipped_keypoints = augmentor.flip(X_train, y_train, False)
h_flipped_images, h_flipped_keypoints = shuffle(h_flipped_images, h_flipped_keypoints, random_state=2)
v_flipped_images, v_flipped_keypoints = shuffle(v_flipped_images, v_flipped_keypoints, random_state=3)

In [0]:
rotation_angles = [12]
rotated_images, rotated_keypoints = augmentor.rotate(X_train, y_train, rotation_angles)
rotated_images, rotated_keypoints = shuffle(rotated_images, rotated_keypoints, random_state=4)

In [0]:
altered_images, altered_keypoints = augmentor.alterBrightness(rotated_images, rotated_keypoints, B=True, D=True)
altered_images, altered_keypoints = shuffle(altered_images, altered_keypoints, random_state=5)

In [0]:
shifts = [12]
shifted_images, shifted_keypoints = augmentor.shiftImage(altered_images, altered_keypoints, shifts=shifts)
shifted_images, shifted_keypoints = shuffle(shifted_images, shifted_keypoints, random_state=6)

In [0]:
noisy_images, noisy_keypoints = augmentor.addNoise(shifted_images, noise_scale=0.002), shifted_keypoints
noisy_images, noisy_keypoints = shuffle(noisy_images, noisy_keypoints, random_state=7)

In [0]:
print("Horizontal:", h_flipped_images.shape, h_flipped_keypoints.shape)
print("Vertical:", v_flipped_images.shape, v_flipped_keypoints.shape)
print("Rotated:", rotated_images.shape, rotated_keypoints.shape)
print("Altered:", altered_images.shape, altered_keypoints.shape)
print("Shifted:", shifted_images.shape, shifted_keypoints.shape)
print("Noisy:", noisy_images.shape, noisy_keypoints.shape)
print("Training Data:", X_train.shape, y_train.shape)

**Adding augmented data to training data**

In [0]:
# new_train_images = np.vstack((X_train, 
#                               h_flipped_images,
#                               rotated_images,
#                               altered_images,
#                               shifted_images,
#                               noisy_images
#                               ))

# new_train_keypoints = np.vstack((y_train,
#                                  h_flipped_keypoints,
#                                  rotated_keypoints,
#                                  altered_keypoints,
#                                  shifted_keypoints,
#                                  noisy_keypoints
#                                  ))

new_train_images = np.vstack((X_train,
                              noisy_images[:int(.10*noisy_images.shape[0])]))
new_train_keypoints = np.vstack((y_train,
                                 noisy_keypoints[:int(.10*noisy_keypoints.shape[0])]))

new_train_images, new_train_keypoints = shuffle(new_train_images, new_train_keypoints, random_state=10)

In [0]:
print(new_train_images.shape)
print(new_train_keypoints.shape)

(4674, 96, 96, 1)
(4674, 30)


#####**Training Model 1**

In [0]:
model = Model_1()
base_learning_rate = 7.5e-5

model.compile(
    loss = 'mse',
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate),
    metrics = ['accuracy'] 
)

In [0]:
epochs = 500
batch_size = 32

history = model.fit(
    new_train_images,
    new_train_keypoints,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.2,
    verbose=2
)

In [0]:
plt.figure()

loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['acc']
val_acc = history.history['val_acc']

plt.plot(loss,linewidth=2, label="training loss")
plt.plot(val_loss, linewidth=2, label="validation loss")

plt.legend()
plt.yscale("log")
plt.xlabel("epoch")
plt.ylabel("log loss")
plt.show()

plt.plot(acc, linewidth=2, label='training accuracy')
plt.plot(val_acc, linewidth=2, label='validation accuracy')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.show()

In [0]:
new_learning_rate = 7.5e-7

model.compile(
    loss = 'mse',
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=new_learning_rate),
    metrics = ['accuracy'] 
)

more_epochs = 500

history = model.fit(
    new_train_images,
    new_train_keypoints,
    epochs=more_epochs+epochs,
    batch_size=batch_size,
    validation_split=0.2,
    verbose=2,
    initial_epoch=history.epoch[-1]+1
)

In [0]:
plt.figure()

loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['acc']
val_acc = history.history['val_acc']

plt.plot(loss,linewidth=2, label="training loss")
plt.plot(val_loss, linewidth=2, label="validation loss")

plt.legend()
plt.yscale("log")
plt.xlabel("epoch")
plt.ylabel("log loss")
plt.show()

plt.plot(acc, linewidth=2, label='training accuracy')
plt.plot(val_acc, linewidth=2, label='validation accuracy')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.show()

**Test Performance**

In [0]:
X_test, _ = utils.loadData()
y_test = model.predict(X_test)

fig = plt.figure(figsize=(10,10))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.1, wspace=0.1)

images = np.random.choice(range(X_test.shape[0]), 16, replace=False)

for i in range(16):
  axis = fig.add_subplot(5,4, i+1, xticks=[], yticks=[])
  utils.plotSample(X_test[images[i]], y_test[images[i]], axis, "Test Prediction {}".format(i+1))

###**Model 2**

####**Defining the Model**

In [0]:
def Model_3():
  model = tf.keras.Sequential()

  model.add(tf.keras.layers.Conv2D(filters=32,
                                   kernel_size=(3,3),
                                   activation='relu',
                                   padding='same',
                                   input_shape=(96,96,1)
                                   ))
  model.add(tf.keras.layers.BatchNormalization())

  model.add(tf.keras.layers.Conv2D(filters=32,
                                   kernel_size=(3,3),
                                   activation='relu',
                                   padding='same'
                                   ))
  model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
  model.add(tf.keras.layers.BatchNormalization())

  model.add(tf.keras.layers.Conv2D(filters=64,
                                   kernel_size=(3,3),
                                   activation='relu',
                                   padding='same'
                                   ))
  model.add(tf.keras.layers.BatchNormalization())

  model.add(tf.keras.layers.Conv2D(filters=64,
                                   kernel_size=(3,3),
                                   activation='relu',
                                   padding='same'
                                   ))
  model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
  model.add(tf.keras.layers.BatchNormalization())

  model.add(tf.keras.layers.Conv2D(filters=96,
                                   kernel_size=(3,3),
                                   activation='relu',
                                   padding='same'
                                   ))
  model.add(tf.keras.layers.BatchNormalization())

  model.add(tf.keras.layers.Conv2D(filters=96,
                                   kernel_size=(3,3),
                                   activation='relu',
                                   padding='same'
                                   ))
  model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
  model.add(tf.keras.layers.BatchNormalization())

  model.add(tf.keras.layers.Conv2D(filters=128,
                                   kernel_size=(3,3),
                                   activation='relu',
                                   padding='same'
                                   ))
  model.add(tf.keras.layers.BatchNormalization())

  model.add(tf.keras.layers.Conv2D(filters=128,
                                   kernel_size=(3,3),
                                   activation='relu',
                                   padding='same'
                                   ))
  model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
  model.add(tf.keras.layers.BatchNormalization())

  model.add(tf.keras.layers.Conv2D(filters=256,
                                   kernel_size=(3,3),
                                   activation='relu',
                                   padding='same'
                                   ))
  model.add(tf.keras.layers.BatchNormalization())
  
  model.add(tf.keras.layers.Conv2D(filters=256,
                                   kernel_size=(3,3),
                                   activation='relu',
                                   padding='same'
                                   ))
  model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
  model.add(tf.keras.layers.BatchNormalization())

  model.add(tf.keras.layers.Conv2D(filters=512,
                                   kernel_size=(3,3),
                                   activation='relu',
                                   padding='same'
                                   ))
  model.add(tf.keras.layers.BatchNormalization())
  
  model.add(tf.keras.layers.Conv2D(filters=512,
                                   kernel_size=(3,3),
                                   activation='relu',
                                   padding='same'
                                   ))
  model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
  model.add(tf.keras.layers.BatchNormalization())

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(512, activation='relu'))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dropout(0.1))
  model.add(tf.keras.layers.Dense(30))

  return model

In [0]:
model = Model_3()
base_learning_rate = 2.5e-6

model.compile(
    loss = 'mse',
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate),
    metrics = ['mae', 'accuracy'] 
)

model.summary()

**Model Architecture**

In [0]:
plot_model(
    model, 
    to_file='model.png', 
    show_layer_names=True, 
    show_shapes=True)

####**Training the Model**

In [0]:
epochs = 500 #epochs before the loss starts increasing
batch_size = 32

history = model.fit(
    new_train_images,
    new_train_keypoints,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.2,
    verbose=2
)

**Learning Rate Check**
A check for the optimal curve, monotonically decreasing loss without regularization

In [0]:
plt.figure()

loss = history.history['loss']
val_loss = history.history['val_loss']
mae = history.history['mean_absolute_error']
val_mae = history.history['val_mean_absolute_error']

plt.plot(loss,linewidth=2, label="training loss")
plt.plot(val_loss, linewidth=2, label="validation loss")

plt.legend()
plt.yscale("log")
plt.xlabel("epoch")
plt.ylabel("log loss")
plt.show()

plt.plot(mae, linewidth=2, label='training MAE')
plt.plot(val_mae, linewidth=2, label='validation MAE')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('MAE')
plt.show()

**Continue training with new learning rate**

In [0]:
more_epochs = 500
new_learning_rate = 2.5e-8

model.compile(
    loss = 'mse',
    optimizer = tf.keras.optimizers.Adam(learning_rate=new_learning_rate),
    metrics = ['mae','accuracy']
)

last_epoch = history.epoch[-1] + 1
history = model.fit(
    new_train_images,
    new_train_keypoints,
    epochs=more_epochs+last_epoch,
    initial_epoch = last_epoch,
    batch_size=batch_size,
    validation_split=0.2,
    verbose=2
)

**Learning Rate Check** A check for the optimal curve, monotonically decreasing loss without regularization

In [0]:
plt.figure()

loss = history.history['loss']
val_loss = history.history['val_loss']
mae = history.history['mean_absolute_error']
val_mae = history.history['val_mean_absolute_error']

plt.plot(loss,linewidth=2, label="training loss")
plt.plot(val_loss, linewidth=2, label="validation loss")

plt.legend()
plt.yscale("log")
plt.xlabel("epoch")
plt.ylabel("log loss")
plt.show()

plt.plot(mae, linewidth=2, label='training MAE')
plt.plot(val_mae, linewidth=2, label='validation MAE')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('MAE')
plt.show()

In [0]:
X_test, _ = utils.loadData()
y_test = model.predict(X_test)

fig = plt.figure(figsize=(10,10))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.1, wspace=0.1)

images = np.random.choice(range(X_test.shape[0]), 16, replace=False)

for i in range(16):
  axis = fig.add_subplot(5,4, i+1, xticks=[], yticks=[])
  utils.plotSample(X_test[images[i]], y_test[images[i]], axis, "Test Prediction {}".format(i+1))

###**Using Missing Data**

#####**Loading Data**

In [0]:
X_train, y_train = utils.loadData('train', drop_null=False)
clean_X_train, clean_y_train = utils.loadData('train', drop_null=True)

In [0]:
print ("Shape of X:",X_train.shape, "Min of X:", X_train.min(), "Max of X:", X_train.max())
print ("Shape of y:",y_train.shape, "Min of y:", y_train.min(), "Max of y:", y_train.max())
print ("Shape of X:",clean_X_train.shape, "Min of X:", clean_X_train.min(), "Max of X:", clean_X_train.max())
print ("Shape of y:",clean_y_train.shape, "Min of y:", clean_y_train.min(), "Max of y:", clean_y_train.max())

**Exploring Dataset**

In [0]:
fig = plt.figure(figsize=(10,10))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.1, wspace=0.1)

images = np.random.choice(range(X_train.shape[0]), 16, replace=False)
for i in range(16):
  axis = fig.add_subplot(5,4, i+1, xticks=[], yticks=[])
  utils.plotSample(X_train[i], y_train[i], axis)

#####**Preparing Augmented Data**

In [0]:
augmentor = DataAugmentation.Augmentor()

In [0]:
h_flipped_images, h_flipped_keypoints = augmentor.flip(clean_X_train, clean_y_train)
v_flipped_images, v_flipped_keypoints = augmentor.flip(clean_X_train, clean_y_train, False)
h_flipped_images, h_flipped_keypoints = shuffle(h_flipped_images, h_flipped_keypoints, random_state=2)
v_flipped_images, v_flipped_keypoints = shuffle(v_flipped_images, v_flipped_keypoints, random_state=3)

In [0]:
rotation_angles = [12]
rotated_images, rotated_keypoints = augmentor.rotate(clean_X_train, clean_y_train, rotation_angles)
rotated_images, rotated_keypoints = shuffle(rotated_images, rotated_keypoints, random_state=4)

In [0]:
altered_images, altered_keypoints = augmentor.alterBrightness(clean_X_train, clean_y_train, B=True, D=True)
altered_images, altered_keypoints = shuffle(altered_images, altered_keypoints, random_state=5)

In [0]:
shifts = [12]
shifted_images, shifted_keypoints = augmentor.shiftImage(clean_X_train, clean_y_train, shifts=shifts)
shifted_images, shifted_keypoints = shuffle(shifted_images, shifted_keypoints, random_state=6)

In [0]:
noisy_images, noisy_keypoints = augmentor.addNoise(clean_X_train, noise_scale=0.002), clean_y_train
noisy_images, noisy_keypoints = shuffle(noisy_images, noisy_keypoints, random_state=7)

In [0]:
print("Horizontal:", h_flipped_images.shape, h_flipped_keypoints.shape)
print("Vertical:", v_flipped_images.shape, v_flipped_keypoints.shape)
print("Rotated:", rotated_images.shape, rotated_keypoints.shape)
print("Altered:", altered_images.shape, altered_keypoints.shape)
print("Shifted:", shifted_images.shape, shifted_keypoints.shape)
print("Noisy:", noisy_images.shape, noisy_keypoints.shape)
print("Training Data:", X_train.shape, y_train.shape)

**Adding augmented data to training data**

In [0]:
new_train_images = np.vstack((X_train, 
                              rotated_images,
                              altered_images,
                              shifted_images,
                              noisy_images
                              ))

new_train_keypoints = np.vstack((y_train,
                                 rotated_keypoints,
                                 altered_keypoints,
                                 shifted_keypoints,
                                 noisy_keypoints
                                 ))

# new_train_images = np.vstack((X_train,
#                               noisy_images[:int(.10*noisy_images.shape[0])]))
# new_train_keypoints = np.vstack((y_train,
#                                  noisy_keypoints[:int(.10*noisy_keypoints.shape[0])]))

new_train_images, new_train_keypoints = shuffle(new_train_images, new_train_keypoints, random_state=10)

In [0]:
print(new_train_images.shape)
print(new_train_keypoints.shape)

#####**Training Model 3**

In [0]:
checkpointer = ModelCheckpoint(filepath = 'model_3_best_10px.hdf5', 
                               monitor='val_mean_absolute_error', 
                               verbose=1, 
                               save_best_only=True, 
                               mode='min')

model = Model_3()
base_learning_rate = 1e-3
batch_size = 32
epochs = 100

model.compile(
    loss = 'mse',
    optimizer = tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
    metrics = ['mae', 'accuracy'] 
)

history = model.fit(new_train_images,
                    new_train_keypoints,
                    epochs=epochs,
                    batch_size = batch_size,
                    callbacks=[checkpointer],
                    validation_split = 0.05)

In [0]:
plt.figure()

loss = history.history['loss']
val_loss = history.history['val_loss']
mae = history.history['mean_absolute_error']
val_mae = history.history['val_mean_absolute_error']
acc = history.history['acc']
val_acc = history.history['val_acc']

plt.plot(loss,linewidth=2, label="training loss")
plt.plot(val_loss, linewidth=2, label="validation loss")

plt.legend()
plt.yscale("log")
plt.xlabel("epoch")
plt.ylabel("log loss")
plt.show()

plt.plot(mae, linewidth=2, label='training MAE')
plt.plot(val_mae, linewidth=2, label='validation MAE')

plt.legend()
plt.xlabel('epochs')
plt.ylabel('MAE')
plt.show()

plt.plot(acc, linewidth=2, label='training acc')
plt.plot(val_acc, linewidth=2, label='validation acc')

plt.legend()
plt.xlabel("epoch")
plt.ylabel("accuracy")
plt.show()

In [0]:
model = Model_3()
model.load_weights('model_3_best_10px.hdf5')

In [0]:
X_test, _ = utils.loadData()
y_test = model.predict(X_test)

fig = plt.figure(figsize=(10,10))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.1, wspace=0.1)

images = np.random.choice(range(X_test.shape[0]), 16, replace=False)

for i in range(16):
  axis = fig.add_subplot(5,4, i+1, xticks=[], yticks=[])
  utils.plotSample(X_test[images[i]], y_test[images[i]], axis, "Test Prediction {}".format(i+1))

#####**Training Model 1**

In [0]:
checkpointer = ModelCheckpoint(filepath = 'model_1_best_15px.hdf5', 
                               monitor='val_mean_absolute_error', 
                               verbose=1, 
                               save_best_only=True, 
                               mode='min')

model = Model_1()
base_learning_rate = 1e-3

model.compile(
    loss = 'mse',
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['mae','accuracy'] 
)

In [40]:
epochs = 100
batch_size = 32

history = model.fit(
    new_train_images,
    new_train_keypoints,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.05,
    callbacks=[checkpointer],
    verbose=1
)

20576/20606 [============================>.] - ETA: 0s - loss: 0.0033 - mean_absolute_error: 0.0427 - acc: 0.7442
Epoch 00011: val_mean_absolute_error improved from 0.03944 to 0.03813, saving model to model_1_best_15px.hdf5
20606/20606 [==============================] - 10s 477us/sample - loss: 0.0033 - mean_absolute_error: 0.0427 - acc: 0.7441 - val_loss: 0.0030 - val_mean_absolute_error: 0.0381 - val_acc: 0.7613
Epoch 12/100
20544/20606 [============================>.] - ETA: 0s - loss: 0.0033 - mean_absolute_error: 0.0427 - acc: 0.7494
Epoch 00012: val_mean_absolute_error improved from 0.03813 to 0.03595, saving model to model_1_best_15px.hdf5
20606/20606 [==============================] - 10s 471us/sample - loss: 0.0033 - mean_absolute_error: 0.0427 - acc: 0.7492 - val_loss: 0.0027 - val_mean_absolute_error: 0.0360 - val_acc: 0.7908
Epoch 13/100
20512/20606 [============================>.] - ETA: 0s - loss: 0.0033 - mean_absolute_error: 0.0426 - acc: 0.7515
Epoch 00013: val_mean_ab

KeyboardInterrupt: ignored

In [0]:
plt.figure()

loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['acc']
val_acc = history.history['val_acc']

plt.plot(loss,linewidth=2, label="training loss")
plt.plot(val_loss, linewidth=2, label="validation loss")

plt.legend()
plt.yscale("log")
plt.xlabel("epoch")
plt.ylabel("log loss")
plt.show()

plt.plot(acc, linewidth=2, label='training accuracy')
plt.plot(val_acc, linewidth=2, label='validation accuracy')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.show()

In [0]:
model = Model_1()
model.load_weights('model_1_best_15px.hdf5')

**Test Performance**

In [0]:
X_test, _ = utils.loadData()
y_test = model.predict(X_test)

fig = plt.figure(figsize=(10,10))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.1, wspace=0.1)

images = np.random.choice(range(X_test.shape[0]), 16, replace=False)

for i in range(16):
  axis = fig.add_subplot(5,4, i+1, xticks=[], yticks=[])
  utils.plotSample(X_test[images[i]], y_test[images[i]], axis, "Test Prediction {}".format(i+1))

##**Creating Kaggle Submit File**

In [0]:
df = pd.read_csv('training.csv')
cols = df.columns[:-1]

y_pred = []
y_test = pd.DataFrame(y_test, columns=cols)
y_pred.append(y_test)
y_pred = pd.concat(y_pred, axis=1)

In [0]:
utils.prepareSubmission(y_pred, 'results')